In [60]:
import h5py
import matplotlib
import matplotlib.pyplot as plt
# from habitat.utils.visualizations import maps
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import os
import cv2
import ast
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm
from PIL import Image

In [61]:
dir_with_hdf5_datasets = '/media/cds-s/data/Datasets/Habitat/test_10_25_depth_0_100/dataset_hdf5/'

In [62]:
T = np.array(dtype=np.float32, object=[
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [1, 0, -1, 0],
    [0, 0, 0, 1]
])
# T = np.eye(4)
T_inv = np.linalg.inv(T)
r = R.from_rotvec(np.pi * np.array([1, 0, 0]))

In [63]:
def action_to_mat44(action, rot_delta, pos_delta):
    if action == 0:
        a = 0
        d = 0
    if action == 1:
        a = 0
        d = pos_delta
    elif action == 2:
        a = -rot_delta
        d = 0
    elif action == 3:
        a = rot_delta
        d = 0
    a = np.pi * a / 180
    d /= 100
    T = np.array([
        [np.cos(a), 0, np.sin(a), 0],
        [0, 1, 0, 0],
        [-np.sin(a), 0, np.cos(a), d],
        [0, 0, 0, 1]
    ])
    return T

In [64]:
for filename_hdf5_dataset in tqdm(os.listdir(dir_with_hdf5_datasets)):
    if filename_hdf5_dataset.find('.hdf5') == -1:
        continue
    rot_delta, pos_delta = map(float, filename_hdf5_dataset.split('_')[1:3])
    
    file = h5py.File(os.path.join(dir_with_hdf5_datasets, filename_hdf5_dataset), 'r')
    output_path = dir_with_hdf5_datasets.rstrip('/')[:dir_with_hdf5_datasets.rstrip('/').rfind('/')]+'/datasets/'+ \
                  filename_hdf5_dataset[:filename_hdf5_dataset.rfind('.')]
    
    rgb = file['rgb']
    depth = file['depth']
    semantic = file['semantic']
    objectgoal = file['objectgoal']
    compass = file['compass']
    gps = file['gps']
    heading = file['heading']
    top_down_map = file['top_down_map']
    fog_of_war_mask = file['fog_of_war_mask']
    agent_map_coord = file['agent_map_coord']
    agent_angle = file['agent_angle']
    mapping = np.array(file['mapping'])
    index_to_title_map = ast.literal_eval(str(np.array(file['index_to_title_map']))[2:-1])
    positions = file['position']
    rotations = file['rotation']
    actions = file['action']
    
    os.makedirs(os.path.join(output_path,'rgb'), exist_ok=True)
    os.makedirs(os.path.join(output_path,'depth'), exist_ok=True)
    depth_file = open(os.path.join(output_path, 'depth.txt'), 'w')
    rgb_file = open(os.path.join(output_path, 'rgb.txt'), 'w')
    gt_filename = '/media/cds-s/data/Datasets/Habitat/gt/'
    gt_filename = gt_filename+filename_hdf5_dataset[:filename_hdf5_dataset.rfind('.')] + '.txt'
    gt_file = open(gt_filename, 'w')
    rgb_file.close()
    depth_file.close()
    gt_file.close()
    depth_file = open(os.path.join(output_path, 'depth.txt'), 'a')
    rgb_file = open(os.path.join(output_path, 'rgb.txt'), 'a')
    gt_file = open(gt_filename, 'a')
    action_file = open(os.path.join(output_path, 'control.txt'), 'w')
    for i in range(3):
        depth_file.write('ignore\n')
        rgb_file.write('ignore\n')
    for i in range(0, len(rgb)):
        cv2.imwrite(os.path.join(output_path, 'rgb', str(i).zfill(6)+'.png'), rgb[i])
        
        depth_image = 100 * depth[i]
        
        cv2.imwrite(os.path.join(output_path, 'depth', str(i).zfill(6)+'.exr'), depth_image)
        rgb_file.write('{:.3f} '.format(i/10))
        rgb_file.write('rgb/'+str(i).zfill(6)+'.png\n')
        depth_file.write('{:.3f} '.format(i/10))
        depth_file.write('depth/'+str(i).zfill(6)+'.exr\n')
        
        X, Y, Z = positions[i]
        w, x, y, z = rotations[i]
        
        x, y, z, w = (R.from_quat([x, y, z, w]) * r).as_quat()
        
#         pose = np.eye(4)
#         pose[:3, :3] = (R.from_quat([x, y, z, w])).as_matrix()
#         pose[:3, 3] = [X, Y, Z]
        
#         pose = T_inv @ pose @ T
#         if i == 3:
#             pose_3 = pose.copy()
#             pose_3_inv = np.linalg.inv(pose_3)
#         pose = pose_3 @ pose @ pose_3_inv
        
#         x, y, z, w = R.from_matrix(pose[:3, :3]).as_quat()
#         X, Y, Z = pose[:3, 3]
        
        gt_file.write('{:.3f} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e}\n'\
              .format(i/10, X, Y, Z, x, y, z, w))
        
#         gt_file.write('{:.3f} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e}\n'\
#                       .format(i/10, -positions[i, 0], positions[i, 1], -positions[i, 2],
#                         rotations[i, 0], rotations[i, 1], rotations[i, 2], rotations[i, 3]))
#         gt_file.write('{:.3f} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e}\n'\
#                       .format(i/10, pose[0, 3], pose[1, 3], pose[2, 3], \
#                               quat_pose[0], quat_pose[1], quat_pose[2], quat_pose[3]))
        
        action_file.write('{:.3f} '.format(i/10))
#         action_file.write('{:d} '.format(actions[i - 1]))
        action_file.write(('{:.8e} ' * 12 + '\n').format(*action_to_mat44(actions[i], rot_delta, pos_delta)[:3].flatten()))
    
    rgb_file.close()
    depth_file.close()
    gt_file.close()
    action_file.close()

100%|██████████| 129/129 [08:23<00:00,  3.90s/it]


In [46]:
file.keys()

<KeysViewHDF5 ['action', 'agent_angle', 'agent_map_coord', 'compass', 'depth', 'fog_of_war_mask', 'gps', 'heading', 'index_to_title_map', 'mapping', 'objectgoal', 'position', 'rgb', 'rotation', 'semantic', 'top_down_map']>

In [23]:
depth[i].min(), depth[i].max()

(0.0, 0.09805029)

In [18]:
np.isfinite(depth_image).sum() / depth_image.shape[0] / depth_image.shape[1]

0.8368701171874999

In [25]:
quat_pose

array([0.        , 0.88368756, 0.        , 0.46807724])

In [27]:
quat_pose.as_quat()

AttributeError: 'numpy.ndarray' object has no attribute 'as_quat'

In [44]:
pose = np.eye(4)
pose[:3, :3] = R.from_quat([rotations[i, 1], rotations[i, 2], rotations[i, 3], rotations[i, 0]]).as_matrix()
pose[:3, 3] = [positions[i, 2], positions[i, 0], positions[i, 1]]

In [50]:
R.from_matrix(pose[:3, :3]).as_quat()

array([ 0.88368756,  0.        ,  0.        , -0.46807724])

In [32]:
pose_3_inv

array([[  0.89345827,   0.        ,  -0.10138863,   4.37532418],
       [  0.        ,   1.        ,   0.        ,   0.18396711],
       [  0.20277726,   0.        ,   1.09623553, -30.26972337],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

In [48]:
file['action'][:]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 3, 1, 1,
       1, 1, 1, 3, 3, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3,
       3, 3, 3, 1, 1, 1, 3, 1, 0])

In [21]:
'metrics' in ['metrics', 'yes']

True

In [24]:
np.array([1,2,3])**2

array([1, 4, 9])

In [66]:
int(False)

0

In [67]:
np.mean([])

/home/cds-s/anaconda3/envs/python37/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/cds-s/anaconda3/envs/python37/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan